This is a code for the quantum addition algorithm that uses the approximate fourier transform, using an algorithm described in the paper "Addition on a quantum computer" cited in our first report.

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import numpy as np


def quantum_addition(qc, a, b, c):
    # apply the approximate Fourier transform to the first register
    for i in range(len(a)):
        qc.h(a[i])
        for j in range(i+1, len(a)):
            qc.cp(2 * np.pi / (2 ** (j-i)), a[j], a[i])

    # apply the controlled phase shift gates to the second register
    for i in range(len(b)):
        qc.cp(2 * np.pi * 2 ** i, a[len(a) - i - 1], b[i])

    # apply the inverse approximate Fourier transform to the first register
    for i in reversed(range(len(a))):
        for j in reversed(range(i+1, len(a))):
            qc.cp(-2 * np.pi / (2 ** (j-i)), a[j], a[i])
        qc.h(a[i])

    # copy the result from the first register to the third register
    qc.cx(a, c)

# create three quantum registers
a = QuantumRegister(3, 'a')  # first input register
b = QuantumRegister(3, 'b')  # second input register
c = QuantumRegister(3, 'c')  # output register

# create a classical register to measure the output register
m = ClassicalRegister(3, 'm')

# create a quantum circuit
qc = QuantumCircuit(a, b, c, m)

# set the inputs
qc.x(a[0])
qc.x(b[1])
qc.x(b[2])

# perform quantum addition
quantum_addition(qc, a, b, c)

# measure the output register
qc.measure(c, m)

# simulate the circuit
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend=backend, shots=1024).result()

# print the results
print(result.get_counts(qc))


{'001': 1024}


In this implementation, we use the approximate Fourier transform instead of the quantum Fourier transform. The approximate Fourier transform is a simplified version of the quantum Fourier transform that uses fewer gates and is therefore easier to implement in hardware. The quantum_addition function now applies the approximate Fourier transform instead of the QFT to the first input register a. We use qc.h() gates and controlled qc.crz() gates to implement the approximate Fourier transform.

The rest of the algorithm is the same as before: we apply controlled phase shift gates to the second input register b, apply the inverse approximate Fourier transform to a, and copy the result from a to the output register c.

The output of the code is a dictionary that maps each possible measurement outcome to the number of times that outcome was observed in the simulation. For example, if the output is {'000': 1024}, it means that the output register measured 000 in all 1024 shots of the simulation.

In this case, since we set the input to be a = 001 and b = 011, the expected output of the quantum addition circuit should be c = 100. That is because a + b = 001 + 011 = 100 in binary.

Let's look at an example output from the simulation. Suppose the output is {'000': 53, '100': 971}. This means that the output register measured 000 in 53 shots and 100 in 971 shots. We can interpret this result as follows: the circuit produced the correct output c = 100 in 971 out of 1024 shots, or approximately 95% of the time. The remaining 5% of the time, the output was 000, which is not the expected output.

Note that due to the probabilistic nature of quantum computing, the actual output of the circuit can vary from one execution to the next, even if the input is the same. However, as we increase the number of shots in the simulation, the probability of observing the correct output approaches 1.